In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

In [2]:
#Hyperparamters
batch_size = 128
epochs = 10
learning_rate = 0.001

###Dataset

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)

train_loader = DataLoader(train_dataset,batch_size = batch_size,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=batch_size)

100%|██████████| 170M/170M [00:03<00:00, 49.2MB/s]


###MLP MODEL

In [4]:
class MLP(nn.Module):
  def __init__(self):
    super(MLP,self).__init__()
    self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(3*32*32,512),
        nn.ReLU(),
        nn.Linear(512,256),
        nn.ReLU(),
        nn.Linear(256,10)
    )

  def forward(self,x):
    return self.model(x)

model = MLP()

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

###Training Loop

In [6]:
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  for inputs , targets in train_loader:
    outputs = model(inputs)
    loss = criterion(outputs,targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f"Epoch {epoch+1}/{epochs},loss: {running_loss:.4f}")

Epoch 1/10,loss: 636.2409
Epoch 2/10,loss: 549.8253
Epoch 3/10,loss: 502.4490
Epoch 4/10,loss: 466.9338
Epoch 5/10,loss: 433.9554
Epoch 6/10,loss: 400.0743
Epoch 7/10,loss: 373.1939
Epoch 8/10,loss: 340.7516
Epoch 9/10,loss: 313.6897
Epoch 10/10,loss: 284.7523


###Evalutaion

In [9]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for inputs,targets in test_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs,1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 53.47%
